# Reading PX-1000 Data and Using Chart

In [ ]:
%matplotlib inline

In [ ]:
import os
import sys
import time
import numpy as np
import urllib.request
import matplotlib
import radarkit
import chart

if not sys.version_info.major is 3:
    print('PyRadarKit requires Python 3')
    raise

In [ ]:
matplotlib.rcParams['font.family'] = 'serif'
matplotlib.rcParams['font.serif'] = ['Arial']
matplotlib.rcParams['font.sans-serif'] = ['System Font', 'Verdana', 'Arial']
matplotlib.rc('text', usetex=False)
matplotlib.rcParams['figure.dpi'] = 144

In [ ]:
# Data filenames
folder = os.path.expanduser('~/Downloads/')
prefix = 'PX-20170220-050706-E2.4'
files = ['{}-{}.nc'.format(prefix, x) for x in ['Z', 'V', 'W', 'D', 'P', 'R']]

# Download data files if not exist
for file in files:
    url = 'http://arrc.ou.edu/~boonleng/files/{}'.format(file)
    dst = '{}/{}'.format(folder, file)
    if not os.path.isfile(dst):
        print('Downloading {} ...'.format(file))
        urllib.request.urlretrieve(url, dst)

In [ ]:
# Read in data files
sweep = radarkit.read('{}/{}-Z.nc'.format(folder, prefix))
if sweep is None:
    print('Unable to continue.\n')
    raise

## Coordinates of the Data Cells

In [ ]:
# Number of gates that we want to show
ng = 1000

# Original range (km) and zimuth (rad) of the data
r = 1.0e-3 * sweep['range'][:ng]
a = sweep['azimuth'] * np.pi / 180.0

# Coordinates
rr, aa = np.meshgrid(r, a)
xx = rr * np.sin(aa)
yy = rr * np.cos(aa)

## Using `chart.showPPI()`

In [ ]:
symbol = 'P'
data = sweep['products'][symbol]['data'][:, :ng]
_ = chart.showPPI(xx, yy, data, style=symbol)

## Generating All Product Figures Inline Using `chart.showPPI()`

In [ ]:
# Loop through all products
for key, product in sweep['products'].items():
    symbol = product['symbol']
    name = product['name']
    unit = product['unit']
    data = product['data'][:, 0:ng]

    #ppi = chart.showPPI(xx, yy, data, title='{} - {} ({})'.format(symbol, name, unit), style=symbol)
    _ = chart.showPPI(xx, yy, data, style=symbol)

## Using `chart.Chart()`

In [ ]:
symbol = 'Z'
data = sweep['products'][symbol]['data'][:, :ng]
fig = chart.Chart(a, r, data, style=symbol)

## Generating All Product Figures Off-Screen and Saving PNGs Using `chart.Chart()`

In [ ]:
for key, product in sweep['products'].items():
    symbol = product['symbol']
    name = product['name']
    unit = product['unit']
    data = product['data'][:, 0:ng]
    # Set figure with new data and symbol
    fig.set_data(data, style=symbol)
    fig.savefig('{}/{}.png'.format(folder, symbol))

## Performance Measurements

In [ ]:
symbol = 'P'
data = sweep['products'][symbol]['data'][:, :ng]

In [ ]:
s = time.time()
fig = chart.Chart(a, r, data, style=symbol)
e = time.time()
print(e - s)

In [ ]:
s = time.time()
ppi = chart.showPPI(xx, yy, data, style=symbol)
e = time.time()
print(e - s)

In [ ]:
s = time.time()
fig.set_data(data, style=symbol)
e = time.time()
print(e - s)
fig.fig